In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import model_from_json, clone_model
import sys

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')

from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression
from ann_framework import aux_functions

from keras import regularizers

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


# Define different the model to test

In [2]:
def RULmodel_SN_1(input_shape):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l1_l2(l1=l1_lambda_regularization, l2=l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l1_l2(l1=l1_lambda_regularization, l2=l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [3]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN_1(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

# Create data Handler

In [4]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Run experiments

In [5]:
#datasets = [1, 2, 3, 4]
datasets = [2]
iterations = 1

window_sizes = {1:24,2:17,3:24,4:17}
strides = {1:1,2:1,3:1,4:1}
max_ruls = {1:129, 2:139, 3:129, 4:139}

scores ={1:[], 2:[], 3:[], 4:[]}
models = {1:RULmodel_SN_1}


#Create tunable model

num_features = len(selected_features)

shape = len(selected_features)*window_size
#file = open("ResultsBestModelAllDatasetsSingleSet.csv", "w")
#csvfile = csv.writer(file, lineterminator='\n')

tempScoresRMSE = []
tempScoresRHS = []
tempTime = []

#Run experiments
for i in datasets:
    
    min_rmse = 1000
    bestModel = None
    bestIndex = 0
    
    print("Working on dataset " + str(i))
        
    tempScoresRMSE = np.zeros((iterations,1))
    tempScoresRHS = np.zeros((iterations,1))
    tempTime = np.zeros((iterations,1))
        
    input_shape = window_sizes[i]*num_features #For simple ANN

    for j in range(iterations):

        #Create keras model
        print("Iteration "+str(j+1))
        #file.write("Iteration "+str(j+1)+'\n\n')

        model = get_compiled_model(input_shape, model_type='ann')
        tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

        #Load data
        min_max_scaler = MinMaxScaler()
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = window_sizes[i]
        tModel.data_handler.sequence_stride = strides[i]
        tModel.data_handler.max_rul = max_ruls[i]

        tModel.data_handler.data_scaler = None
        tModel.data_scaler = min_max_scaler

        tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
        #tModel.data_handler.print_data()
        #tModel.print_data()

        #Train model
        tModel.epochs = 200

        tModel.train_model(verbose=1)
        tModel.evaluate_model(['rhs', 'rmse'], round=2)

        cScores = tModel.scores
        rmse = math.sqrt(cScores['score_1'])
        rmse2 = cScores['rmse']
        rhs = cScores['rhs']
        time = tModel.train_time

        if rmse2 < min_rmse:
            best_model_weights = tModel.model.get_weights()
            #bestModel.set_weights(tModel.model.get_weights())
            bestIndex = j
            min_rmse = rmse2

        tempScoresRMSE[j] = rmse2
        tempScoresRHS[j] = rhs
        tempTime[j] = time

    print(tempScoresRMSE)
    print('Best Score')
    print(min_rmse)
    
    #save best model to file
    #serialize model to JSON

    #print(best_model_weights)
    model = get_compiled_model(input_shape, model_type='ann')
    bestModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps).model
    bestModel.set_weights(best_model_weights)
    model_json = bestModel.to_json()
    with open("bestRULModel_dataset_"+str(i)+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    bestModel.save_weights("bestRULModel_dataset_"+str(i)+".h5")
    print("Saved best model for dataset "+str(i)+" to disk")

Working on dataset 2
Iteration 1
Loading data for the first time
Reloading data due to parameter change
Loading data for dataset 2 with window_size of 17, stride of 1 and maxRUL of 139. Cros-Validation ratio 0
Loading data from file and computing dataframes
training without cv
Epoch 1/200
49599/49599 [==============================] - 1s 19us/step - loss: 5841.2071 - mean_squared_error: 5786.4373
Epoch 2/200
49599/49599 [==============================] - 1s 11us/step - loss: 2381.8143 - mean_squared_error: 2323.4577
Epoch 3/200
49599/49599 [==============================] - 0s 9us/step - loss: 2355.4941 - mean_squared_error: 2298.4184
Epoch 4/200
49599/49599 [==============================] - 0s 9us/step - loss: 2321.4350 - mean_squared_error: 2264.3624
Epoch 5/200
49599/49599 [==============================] - 0s 7us/step - loss: 2283.8800 - mean_squared_error: 2226.1436
Epoch 6/200
49599/49599 [==============================] - 0s 8us/step - loss: 2243.5150 - mean_squared_error: 2184

49599/49599 [==============================] - 0s 7us/step - loss: 907.3118 - mean_squared_error: 822.1058
Epoch 67/200
49599/49599 [==============================] - 0s 10us/step - loss: 909.1481 - mean_squared_error: 824.2821
Epoch 68/200
49599/49599 [==============================] - 0s 7us/step - loss: 900.7474 - mean_squared_error: 816.1663
Epoch 69/200
49599/49599 [==============================] - 0s 8us/step - loss: 897.3076 - mean_squared_error: 813.0164
Epoch 70/200
49599/49599 [==============================] - 0s 7us/step - loss: 897.0692 - mean_squared_error: 813.0998
Epoch 71/200
49599/49599 [==============================] - 0s 7us/step - loss: 896.0657 - mean_squared_error: 812.4116
Epoch 72/200
49599/49599 [==============================] - 0s 7us/step - loss: 900.8913 - mean_squared_error: 817.5157
Epoch 73/200
49599/49599 [==============================] - 0s 6us/step - loss: 895.5451 - mean_squared_error: 812.5121
Epoch 74/200
49599/49599 [==========================

49599/49599 [==============================] - 0s 7us/step - loss: 868.5956 - mean_squared_error: 799.8294
Epoch 135/200
49599/49599 [==============================] - 0s 7us/step - loss: 861.7372 - mean_squared_error: 793.1236
Epoch 136/200
49599/49599 [==============================] - 0s 5us/step - loss: 861.6162 - mean_squared_error: 793.2142
Epoch 137/200
49599/49599 [==============================] - 0s 6us/step - loss: 864.1349 - mean_squared_error: 795.8942
Epoch 138/200
49599/49599 [==============================] - 0s 7us/step - loss: 860.9745 - mean_squared_error: 792.8851
Epoch 139/200
49599/49599 [==============================] - 0s 5us/step - loss: 862.5823 - mean_squared_error: 794.6956
Epoch 140/200
49599/49599 [==============================] - 0s 6us/step - loss: 860.9445 - mean_squared_error: 793.2449
Epoch 141/200
49599/49599 [==============================] - 0s 7us/step - loss: 863.6828 - mean_squared_error: 796.1412
Epoch 142/200
49599/49599 [===================

Saved best model for dataset 2 to disk


In [8]:
# load json and create model
json_file = open('bestRULModel_dataset_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("bestRULModel_dataset_2.h5")
print("Loaded model from disk")


#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Create and compile the models
#nFeatures = len(selected_features)
#shapeSN = nFeatures*windowSize
#modelRULSN = RULmodel_SN_1(shapeSN)
loaded_model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
min_max_scaler = MinMaxScaler()

tModel = SequenceTunableModelRegression('ModelRUL_SN_1', loaded_model, lib_type='keras', data_handler=dHandler_cmaps)

i = 2

tModel.data_handler.change_dataset(i)
tModel.data_handler.sequence_length = window_sizes[i]
tModel.data_handler.sequence_stride = strides[i]
tModel.data_handler.max_rul = max_ruls[i]

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.print_data()
#print(tModel.model.get_weights())
tModel.get_model_description()



Loaded model from disk
Loading data for the first time
Reloading data due to parameter change
Loading data for dataset 2 with window_size of 17, stride of 1 and maxRUL of 139. Cros-Validation ratio 0
Loading data from file and computing dataframes
Description for model: ModelRUL_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                4780      
_________________________________________________________________
fc2 (Dense)                  (None, 20)                420       
_________________________________________________________________
out (Dense)                  (None, 1)                 21        
Total params: 5,221
Trainable params: 5,221
Non-trainable params: 0
_________________________________________________________________


In [9]:
# evaluate loaded model on test data
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
print(cScores)
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

259/259 [==============================] - 0s 512us/step
scores
{'loss': 1221.6244099149374, 'score_1': 1162.0175140263032, 'rhs': 159.51125585952417, 'rmse': 34.037340385825395}
{'loss': 1221.6244099149374, 'score_1': 1162.0175140263032, 'rhs': 159.51125585952417, 'rmse': 34.037340385825395}
RMSE: 34.088377990545446
RMSE2: 34.037340385825395
RHS: 159.51125585952417
Time : None seconds
